# IUM projekt - etap 2
### Jakub Grzechociński  
### Piotr Sawicki


## Treść zadania
> Są osoby, które wchodzą na naszą stronę i nie mogą się zdecydować, którym produktom przyjrzeć się nieco lepiej. Może dało by się im coś polecić?

## Streszczenie etapu pierwszego
Do stworzenia jest system oferujący usługę rekomendacji produktów dla klientów sklepu internetowego. System opiera swoje działanie na danych przejrzeń i zakupów produktów przez użytkowników sklepu oraz na danych samych produktów.

System oferuje rekomendacje na podstawie dwóch oddzielnych modeli:
- `popularnościowy` bazujący na względnej popularności produktów i kategorii oraz preferencji użytkownika
- `collaborative filtering` odtwarzający podobieństwa między klientami

W ramach implementacji systemu należy utworzyć interfejs dla modeli rekomendacji w postaci mikroserwisu. W wyniku jego działania aplikacje klienckie mogą uzyskiwać rekomendacje produktów na podstawie id użytkownika. System pozwala na działanie z wykorzystaniem jednego z dwóch modeli lub w trybie testu A/B dla porównania modeli.

W etapie pierwszym wykonana została analiza danych, w ramach której zidentyfikowane zostały problemy oraz oszacowano, że system jest w stanie skutecznie rekomendować na podstawie danych.

## Zaimplementowane modele

### Model popularnościowy
Model opiera swoje działanie na względnej popularności przedmiotów oraz ich kategorii. Zliczana jest ilość wyświetlneń oraz zakupów każdego z produktów, przy czym zakup ma mnożnik X9 (jest to jeden z hiperparametrów modelu). Produkty są sortowane zgodnie z tak oszacowaną popularnością wewnątrz swoich karegorii. 

## Pokaz działania 

Serwis został umieszczony na maszynie w Oracle Cloud pod publicznym adresem ip `130.61.188.211`. Można więc przetestować jego działanie z dowolnego miejsca. Serwis odpowiada na odpowiednio sformułowane żądanie HTTP.   

Np. Aby uzyskać otrzymać odpowiedź zawierającą zestaw dziesięciu rekomendacji dla klienta o id `120` wystarczy wysłać zapytanie HTTP GET na adres: 
``` 
 130.61.188.211/recommendations?user_id=121&num=10 
``` 
W linii poleceń można szybko wykonać takie zapytanie przy pomocy narzędzia `curl`: 
``` 
curl -i "130.61.188.211/recommendations?user_id=121&num=10" 
``` 
Odpowiedź serwera: 
``` 
HTTP/1.1 200 OK 
Content-Type: application/json 
Date: Sun, 28 Mar 2021 16:30:32 GMT 
Content-Length: 110 
  
{"type":"user","id":121,"attributes":{"recommendations":[1077,1076,1080,1095,1004,1005,1006,1007,1008,1009]}} 
``` 

Serwer chodzący pod publicznym IP pracuje w trybie testu A/B. Przyporządkowanie użytkowników do modelu zapisywane jest w bazie danych SQLite. Po stronie serwera możemy sprawdzić, który system wygenerował ten zestaw rekomendacji: 
``` 
sqlite> SELECT * FROM user_to_model WHERE user_id == 121; 
user_id|model_id 
121|2 
``` 
Następnie szybki wgląd w przypisanie `model_id` do zrozumiałej nazwy: 
``` 
sqlite> SELECT * FROM models; 
model_id|name|user_count 
1|Popularity|4 
2|Collaborative|4 
``` 

Dla zapytania dla użytkowinika o `user_id` 121 otrzymaliśmy więc rekomendacje od systemu `Collaborative`, czyli modelu CF. Na podstawie tak zbieranych danych oraz dalszych danych zbieranych przez sklep internetowy można oszacować skuteczność obu modeli. 

Warto by było teraz sprawdzić jakie produkty zostały zarekomendowane. Są to produkty o `product_id`: 
```
1077,1076,1080,1095,1004,1005,1006,1007,1008,1009 
``` 

In [23]:
import datetime
import numpy as np
import pandas as pd

data_dir = 'data/'
data_raw_dir = 'data_raw/'
products_file = 'products.jsonl'
sessions_file = 'sessions.jsonl'
products_filepath = data_raw_dir + products_file
sessions_filepath = data_dir + sessions_file
products_data = pd.read_json(products_filepath, convert_dates=False, lines=True)
products_data = products_data.set_index('product_id')
sessions_data = pd.read_json(sessions_filepath, convert_dates=False, lines=True)

In [20]:
product_list = [1077,1076,1080,1095,1004,1005,1006,1007,1008,1009]
products_data[products_data.index.isin(product_list)]

,product_name,category_path,price
product_id,,,
1004,Fallout 3 (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,49990000.00
1005,Szalone Króliki Na żywo i w kolorze (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,49.99
1006,Call of Duty 4 Modern Warfare (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,59.90
1007,Dead Space 3 (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,-89.99
1008,Tom Clancy&#39;s Rainbow Six Vegas (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,49.99
1009,Kinect Joy Ride (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,69.00
1076,Samsung CLX-6260FR ### Gadżety Samsung ### Eks...,Komputery;Drukarki i skanery;Biurowe urządzeni...,2399.00
1077,Kyocera FS-C2026MFP,Komputery;Drukarki i skanery;Biurowe urządzeni...,3777.00
1080,Kyocera FS-3140MFP,Komputery;Drukarki i skanery;Biurowe urządzeni...,5301.90


In [24]:
target_user_id = 121
# Odrzucenie danych nie związanych z użytkownikiem o id 121
sessions_data = sessions_data.loc[sessions_data['user_id']==target_user_id]
sessions_data
# Złączenie danych sesji i produktów
#session_join_products = sessions_data.join(products_data.set_index('product_id'), on='product_id', how='left')
#session_join_products = session_join_products[['session_id', 'user_id', 'event_type', 'category_path']]
#session_join_products

,session_id,timestamp,user_id,product_id,event_type,offered_discount,purchase_id
8690,101060,2021-02-10T12:47:11,121.0,1017.0,VIEW_PRODUCT,0,NaN
8691,101060,2021-02-10T12:49:40,121.0,NaN,VIEW_PRODUCT,0,NaN
8692,101060,2021-02-10T12:50:00,121.0,1070.0,VIEW_PRODUCT,0,NaN
8693,101060,2021-02-10T12:52:32,121.0,NaN,VIEW_PRODUCT,0,NaN
8694,101060,2021-02-10T12:57:24,121.0,1039.0,VIEW_PRODUCT,0,NaN
...,...,...,...,...,...,...,...
9379,101144,2021-01-08T08:34:41,121.0,1311.0,VIEW_PRODUCT,0,NaN
9380,101144,2021-01-08T08:36:18,121.0,1300.0,VIEW_PRODUCT,0,NaN
9381,101144,2021-01-08T08:40:59,121.0,1309.0,VIEW_PRODUCT,0,NaN
9382,101145,2021-01-16T21:08:14,121.0,NaN,VIEW_PRODUCT,20,NaN
